In [1]:
import pandas as pd
import plotly.express as px  # (version 4.7.0 or higher)
import plotly.graph_objects as go
import dash
print(dash.__version__)
from dash import Dash, html, dcc
# import dash_core_components as dcc 
# import dash_html_components as html
from dash.dependencies import Input, Output, State
import datetime

# from visualize import Visualize2
import random
# import google
# from google.cloud import bigquery

# from google.oauth2 import service_account
import pandas as pd
import numpy as np
from urllib.request import urlopen
import json

2.7.0


In [2]:
# credentials = service_account.Credentials.from_service_account_file(
# 'chain-coders-5d47d58b2c33.json')

# project_id = 'chain-coders'
# client = bigquery.Client(credentials= credentials,project=project_id)
# query = "SELECT * FROM `redfin.redfin-data-county-fips`"
# df = client.query(query).to_dataframe()

In [3]:
app = dash.Dash(__name__)

In [4]:
# -- Import and clean data (importing csv into pandas)
df = pd.read_csv("redfin-data-county-fips.csv") 

#convert county fips codes to string type to preserve leading 0
df.county_fips = df.county_fips.astype(str) #convert to string
df['county_fips'] = df['county_fips'].str.zfill(5)

#format data type for time
df['period_begin'] = pd.to_datetime(df['period_begin'], format="%Y/%m/%d").dt.date
df['period_end'] = pd.to_datetime(df['period_end'], format="%Y/%m/%d").dt.date

#property types for drop down
property_set = list(set(df['property_type'].values.tolist()))

#metric types for dropdown
metric_set = ['median_sale_price', 'median_ppsf', 'median_sale_price_mom', 'median_ppsf_mom']

#dates for slider
sorted_dates = np.sort(df['period_end'].unique())
sorted_dates = ['{}'.format(i) for i in sorted_dates] #convert each date to string within a list
counter = 0
year = 2012
slider_labels = {}
#labels for slider
for index, date in enumerate(sorted_dates):
       if counter == index:
              slider_labels['{}'.format(index)] = year
              year += 1
              counter += 12
       else: 
              slider_labels['{}'.format(index)] = ''

# labels = {index: date for index, date in enumerate(sorted_dates)} #labels of dates for slider object

#color-scale
color_scale = [[0.0, '#edf8fb'],[0.2, '#ccece6'],[0.4, '#99d8c9'],
       [0.6, '#66c2a4'],[0.8, '#2ca25f'],[1.0, '#006d2c']] # purples


In [5]:
#get geojson for counties
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties_geojson = json.load(response)

In [6]:
print(slider_labels)


{'0': 2012, '1': '', '2': '', '3': '', '4': '', '5': '', '6': '', '7': '', '8': '', '9': '', '10': '', '11': '', '12': 2013, '13': '', '14': '', '15': '', '16': '', '17': '', '18': '', '19': '', '20': '', '21': '', '22': '', '23': '', '24': 2014, '25': '', '26': '', '27': '', '28': '', '29': '', '30': '', '31': '', '32': '', '33': '', '34': '', '35': '', '36': 2015, '37': '', '38': '', '39': '', '40': '', '41': '', '42': '', '43': '', '44': '', '45': '', '46': '', '47': '', '48': 2016, '49': '', '50': '', '51': '', '52': '', '53': '', '54': '', '55': '', '56': '', '57': '', '58': '', '59': '', '60': 2017, '61': '', '62': '', '63': '', '64': '', '65': '', '66': '', '67': '', '68': '', '69': '', '70': '', '71': '', '72': 2018, '73': '', '74': '', '75': '', '76': '', '77': '', '78': '', '79': '', '80': '', '81': '', '82': '', '83': '', '84': 2019, '85': '', '86': '', '87': '', '88': '', '89': '', '90': '', '91': '', '92': '', '93': '', '94': '', '95': '', '96': 2020, '97': '', '98': '', '

In [7]:
df.columns

Index(['period_begin', 'period_end', 'period_duration', 'region_type',
       'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region',
       'city', 'state_x', 'state_code', 'property_type', 'property_type_id',
       'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy',
       'median_list_price', 'median_list_price_mom', 'median_list_price_yoy',
       'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf',
       'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold',
       'homes_sold_mom', 'homes_sold_yoy', 'pending_sales',
       'pending_sales_mom', 'pending_sales_yoy', 'new_listings',
       'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom',
       'inventory_yoy', 'months_of_supply', 'months_of_supply_mom',
       'months_of_supply_yoy', 'median_dom', 'median_dom_mom',
       'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom',
       'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom',

In [8]:
print(sorted_dates)
print(len(sorted_dates))

['2012-01-31', '2012-02-29', '2012-03-31', '2012-04-30', '2012-05-31', '2012-06-30', '2012-07-31', '2012-08-31', '2012-09-30', '2012-10-31', '2012-11-30', '2012-12-31', '2013-01-31', '2013-02-28', '2013-03-31', '2013-04-30', '2013-05-31', '2013-06-30', '2013-07-31', '2013-08-31', '2013-09-30', '2013-10-31', '2013-11-30', '2013-12-31', '2014-01-31', '2014-02-28', '2014-03-31', '2014-04-30', '2014-05-31', '2014-06-30', '2014-07-31', '2014-08-31', '2014-09-30', '2014-10-31', '2014-11-30', '2014-12-31', '2015-01-31', '2015-02-28', '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31', '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31', '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31', '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31', '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31', '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31', '2017-09-30', '2017-10-31', '2017-11-30', '2017

In [9]:
#read in dummy data
df2 = pd.read_csv("dummy_data2.csv")
df2['county_code'] = df2['county_code'].astype(str) #convert county code as string from int
df2.head(10)

,year,county_code,median_ppsf,median_sale_price,property
0,2023,27053,483,274053,Condo/Co-op
1,2024,27053,38,271021,Condo/Co-op
2,2025,27053,700,419935,Condo/Co-op
3,2026,27053,27,482962,Condo/Co-op
4,2023,27053,685,223464,All Residential
5,2024,27053,667,360849,All Residential
6,2025,27053,552,716118,All Residential
7,2026,27053,93,210531,All Residential
8,2023,27053,526,387056,Townhouse
9,2024,27053,280,402558,Townhouse


In [10]:
df2.dtypes

year                  int64
county_code          object
median_ppsf           int64
median_sale_price     int64
property             object
dtype: object

In [11]:
# ------------------------------------------------------------------------------
# App layout
#what goes inside dash layout are the dash components and any html we need
app.layout = html.Div([

    html.H1("Redfin Choropleth Data", style={'text-align': 'center'}), #html Title

    html.Div(id='output_container', children=["Metric"], style = {'font-weight': 'bold'}),

    #components, e.g., drop downs, sliders
    #first drop down by metric
    dcc.Dropdown(id="metric",
                 placeholder = "Select Metric",
                 options=metric_set,
                 value=metric_set[0],
                 style={'width': "60%",}
                 ),

    html.Br(), #space between dropdowns

    html.Div(id='output_container', style = {'font-weight': 'bold'}, 
    children = ["Property Type"]
    ),

    #second drop down for property type
    dcc.Dropdown(id="property_type",
                 placeholder = "Select Property Type",
                 options=property_set,
                 value=property_set[0],
                 style={'width': "60%"},
                 ),

    
    html.Br(), #space
    
    #create div element, e.g., text to display dropdown selection
    html.Div(id='output_container', children=[]),

    html.Br(), #space

    #graph object, e.g., choropleth
    dcc.Graph(id='choropleth', figure={}), 

    #slider object
    dcc.Slider(0, len(sorted_dates), step = None, #min value of 0 and max of number of unique dates
               value=0, #where the slider starts
               marks = slider_labels,
               tooltip={"placement": "bottom", "always_visible": True}, #place slider at bottom of graph
               id='date_chosen'),
    
    #add line chart
    dcc.Graph(id = "line_chart", figure = {})

])

In [12]:
# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    Output(component_id='output_container', component_property='children'),
    Output(component_id='choropleth', component_property='figure'),
    Output(component_id = 'line_chart', component_property = 'figure'),
    Input(component_id='property_type', component_property='value'),
    Input(component_id = 'metric', component_property = 'value'),
    Input(component_id='date_chosen', component_property='value'),
    Input(component_id = 'choropleth', component_property = 'hoverData'))


def update_graph(property_type, metric, date_chosen, hover):
    print(property_type)
    print(type(property_type))

    container = "Date Selected: {}".format(sorted_dates[date_chosen])

    dff = df.copy()
    dff = dff[['period_begin', 'period_end', 'region', 'median_sale_price', 'median_sale_price_mom', 
    'median_ppsf', 'median_ppsf_mom', 'property_type', 'county_fips']]
    dff = dff[dff['property_type'] == property_type]
    dff = dff[dff['period_end'] == pd.to_datetime(sorted_dates[date_chosen], format="%Y/%m/%d")]

    # Plotly Express
    fig = px.choropleth(
        data_frame = dff,
        geojson = counties_geojson,
        locations='county_fips',
        scope="usa",
        color='{}'.format(metric),
        hover_data=['region', '{}'.format(metric), 'county_fips', 'region'],
        color_continuous_scale= color_scale
    )

    #to generate line chart based on hover data
    if type(hover) == dict:
        print(hover)
        hover_county = hover['points'][0]['customdata'][2]
        hover_county_name = hover['points'][0]['customdata'][3]
        hover_df = df2[(df2['property'] == property_type) & (df2['county_code'] == hover_county)]
        line_data = px.line(hover_df, x = "year", y = '{}'.format(metric), title = 'Predicted {fmetric} for {fcounty}'.format(fmetric = metric, fcounty = hover_county_name))
        result = line_data
        result.update_layout(xaxis = dict(
            dtick = 1
        ))
        
    else:
        result = []
    

    return container, fig, result

In [13]:
if __name__ == '__main__':
    app.run_server(host= '0.0.0.0',port=8050, debug=False)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8050
 * Running on http://172.17.0.2:8050
Press CTRL+C to quit


Exception on /_reload-hash [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/site-packages/flask/app.py", line 1818, in full_dispatch_request
    rv = self.preprocess_request()
  File "/usr/local/lib/python3.7/site-packages/flask/app.py", line 2309, in preprocess_request
    rv = self.ensure_sync(before_func)()
  File "/usr/local/lib/python3.7/site-packages/dash/dash.py", line 1307, in _setup_server
    _validate.validate_layout(self.layout, self._layout_value())
  File "/usr/local/lib/python3.7/site-packages/dash/_validate.py", line 411, in validate_layout
    """
dash.exceptions.DuplicateIdError: Duplicate component id found in the initial layout: `output_container`


172.17.0.1 - - [03/Dec/2022 17:18:09] "GET /_reload-hash HTTP/1.1" 500 -
172.17.0.1 - - [03/Dec/2022 17:18:16] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:18:16] "GET /_dash-dependencies HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:18:16] "GET /_dash-layout HTTP/1.1" 200 -


Townhouse
<class 'str'>


...

172.17.0.1 - - [03/Dec/2022 17:19:30] "POST /_dash-update-component HTTP/1.1" 200 -


{'points': [{'curveNumber': 0, 'pointNumber': 77, 'pointIndex': 77, 'location': '15001', 'z': 485750, 'bbox': {'x0': 808.8477061852592, 'x1': 808.8477061852592, 'y0': 616.5625588549067, 'y1': 616.5625588549067}, 'customdata': ['Hawaii County', 485750, '15001', 'Hawaii County']}]}


172.17.0.1 - - [03/Dec/2022 17:19:31] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:20:38] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:22:46] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:25:21] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:29:37] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:32:44] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:38:04] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:43:24] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:48:44] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:54:04] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 17:59:25] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 18:04:49] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 18:10:14] "GET /_reload-hash HTTP/1.1" 200 -
172.17.0.1 - - [03/Dec/2022 18:15:38] "G

Townhouse
<class 'str'>
All Residential
<class 'str'>


172.17.0.1 - - [04/Dec/2022 05:21:27] "POST /_dash-update-component HTTP/1.1" 200 -
172.17.0.1 - - [04/Dec/2022 05:21:28] "POST /_dash-update-component HTTP/1.1" 200 -


# Experiments

In [ ]:
# df_copy = df.copy()
# df_copy = df_copy[['period_begin', 'period_end', 'county_fips', 'median_sale_price']]
# df_copy['period_begin'] = pd.to_datetime(df_copy['period_begin'], format="%Y/%m/%d").dt.date
# df_copy['period_end'] = pd.to_datetime(df_copy['period_end'], format="%Y/%m/%d").dt.date
# df_2016 = df_copy[df_copy["period_end"] == datetime.date(2016,1,31)]
# df_2016.head(5)

In [ ]:
# fig = px.choropleth(df_2016, geojson = counties_geojson, locations = 'county_fips', color = 'median_sale_price',
#                                     color_continuous_scale="Viridis",
#                                     range_color=(0, 1000000),
#                                     scope="usa",
#                                     labels={'median_sale_price':'median sale price'}
#                                     )

# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# fig.show()